Different code cells for different data processing

In [4]:
import random
from icalendar import Calendar, Event
from datetime import datetime, timedelta
import pytz

# Function to generate random protocol instructions
def generate_protocol(file_path):
    def generate_instruction():
        x = random.randint(1, 5)  # Random number between 1-5
        z = random.randint(1, 120)  # Random number between 1-120
        y = random.choice(["jumping jacks", "push-ups", "sit-ups", "squats", "lunges", "sunlight viewing", "doomscrolling"])  # Random activity
        return f"{x} times a week, for {z} minutes, do {y}"

    with open(file_path, 'w') as file:
        for _ in range(5):  # Generate 5 instructions for each protocol
            instruction = generate_instruction()
            file.write(instruction + '\n')

In [5]:
for i in range(1, int(input("Enter amount of Protocols:"))+1):
    protocol_file_path = f'Data/random_protocol{i}.txt'
    generate_protocol(protocol_file_path)

In [6]:
import math
import re

def round_up_to_ten(n):
    """Round up a number to the nearest ten."""
    return math.ceil(n / 10) * 10

def process_task_line(line):
    """Process a line from the file and return a list of tasks."""
    # Use regular expression to extract the numbers and task name
    match = re.match(r"(\d+) times a week, for (\d+) minutes, do (.+)", line.strip())
    if not match:
        return []

    frequency = int(match.group(1))
    duration = int(match.group(2))
    task_name = match.group(3).strip()

    # Round up the duration and replicate the task
    rounded_duration = round_up_to_ten(duration)
    return [(rounded_duration, task_name) for _ in range(frequency)]

def read_tasks_from_file(file_path):
    """Read tasks from a file and return an array of tasks."""
    tasks = []
    with open(file_path, 'r') as file:
        for line in file:
            tasks.extend(process_task_line(line))
    return tasks




In [8]:
file_path = 'Data/random_protocol1.txt'  # Protocol that gets processed
tasks_array = read_tasks_from_file(file_path)
print(tasks_array)


[(20, 'sunlight viewing'), (50, 'jumping jacks'), (50, 'jumping jacks'), (50, 'jumping jacks'), (50, 'jumping jacks'), (20, 'push-ups'), (20, 'push-ups'), (20, 'push-ups'), (20, 'push-ups'), (70, 'squats'), (70, 'squats'), (70, 'squats'), (70, 'squats'), (70, 'squats'), (110, 'jumping jacks'), (110, 'jumping jacks'), (110, 'jumping jacks')]


Only take in reoccuring schedule in calendar


In [9]:
from icalendar import Calendar, Event
import datetime
import numpy as np
import re

def parse_ical_file(file_path):
    with open(file_path, 'r') as f:
        ical_content = f.read()
    return Calendar.from_ical(ical_content)

def is_recurring(event):
    return 'rrule' in event or 'rdate' in event

def is_within_week(event, week_start):
    event_start = event.get('dtstart').dt
    event_end = event.get('dtend').dt
    week_end = week_start + datetime.timedelta(days=7)
    return week_start <= event_start.date() < week_end

def process_event(event, week_start, calendar_matrix):
    event_start = event.get('dtstart').dt
    event_end = event.get('dtend').dt

    # Adjust time to 8:00-22:00 window
    start_time = max(event_start.time(), datetime.time(8, 0))
    end_time = min(event_end.time(), datetime.time(22, 0))

    # Calculate day and slot indices
    day_index = (event_start.date() - week_start).days
    start_slot = ((start_time.hour - 8) * 6) + (start_time.minute // 10)
    end_slot = ((end_time.hour - 8) * 6) + (end_time.minute // 10)

    # Mark slots as occupied
    for slot in range(start_slot, end_slot):
        if 0 <= day_index < 7 and 0 <= slot < 84:
            calendar_matrix[day_index][slot] = 1

def ical_to_matrix(ical_file, week_start):
    cal = parse_ical_file(ical_file)
    calendar_matrix = np.zeros((7, 84), dtype=int)

    for component in cal.walk():
        if isinstance(component, Event):
            if is_recurring(component) and is_within_week(component, week_start):
                process_event(component, week_start, calendar_matrix)

    return calendar_matrix

# Example usage
ical_file = 'Data/calendar.ics'
week_start = datetime.date(2023, 11, 20)  # Example week start
matrix = ical_to_matrix(ical_file, week_start)
print(matrix)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0